In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import pathlib
import tensorflow as tf
from tensorflow import keras
import os
from tqdm import tqdm
import time

In [ ]:
base_dir = '/kaggle/input/data-science-bowl-2018/'
unzip_base_dir = '/jkaggle/working/'
stage_train_zip = base_dir + 'stage1_train.zip'
stage_train_labels_zip = base_dir + 'stage1_train_labels.csv.zip'
stage_train_unzip = unzip_base_dir + 'stage1_train/'
stage_train_labels_unzip = unzip_base_dir +'stage1_train_labels/'

In [ ]:
import zipfile
for path , unzip_path in zip([stage_train_zip,stage_train_labels_zip],[stage_train_unzip,stage_train_labels_unzip]):
    print(path)
    print(unzip_path)
    print('------------')
    with zipfile.ZipFile(path,'r') as zip_ref:
        zip_ref.extractall(unzip_path)

In [ ]:
test_data = r"/kaggle/input/data-science-bowl-2018/stage1_test.zip"
test_data_unzip = unzip_base_dir + 'stage1_test/'
with zipfile.ZipFile(test_data,'r') as zip_ref:
    zip_ref.extractall(test_data_unzip)

In [ ]:
test = pathlib.Path(test_data_unzip)
lsttest_files = glob.glob(str(test/'*/'))
lsttest_files[:10]

In [ ]:
def get_image(img_path):
    temp_path = glob.glob(img_path+'/images/*')
    img = tf.io.read_file(temp_path[0])
    img = tf.io.decode_image(img)
    img = tf.image.resize(img, (128,128))
  
    arr = img[:, :, :3].numpy()
    new_arr = ((arr - arr.min()) * (1/(arr.max() - arr.min())))
    return new_arr

test_images = []
for i in lsttest_files:
    test_images.append(get_image(i))

In [ ]:
test_dataset = tf.convert_to_tensor(np.array(test_images))

In [ ]:
tf.shape(test_dataset)

In [ ]:
path = pathlib.Path(stage_train_unzip)
lst_files = glob.glob(str(path/'*/'))

In [ ]:
BATCH_SIZE = 64

In [ ]:
class create_ds(keras.utils.Sequence):
    def __init__(self,lst_files,batch_size):
        self.lst_files = lst_files
        self.batch_size = batch_size
        
    def __len__(self):
        return len(self.lst_files)//self.batch_size
    
    def get_image(self,image_path):
        temp_path = glob.glob(image_path+'/images/*')
        img = tf.io.read_file(temp_path[0])
        img = tf.io.decode_image(img)
        img = tf.image.resize(img,(128,128))
        arr = img[:,:,:3].numpy()
        new_arr = ((arr-arr.min()))*(1/(arr.max()-arr.min()))
        
        ma = glob.glob(image_path+'/masks/*')
        mask_ = tf.zeros(shape = (128,128,1))
        
        
        for mask_path in ma :
            mask = tf.io.read_file(mask_path)
            mask = tf.io.decode_image(mask)
            mask = tf.image.resize(mask,(128,128))
            
            mask_ = tf.maximum(mask_,mask)
        return new_arr,mask_.numpy()    
    def __getitem__(self,idx):
        batch = self.lst_files[idx*self.batch_size : (idx+1)*self.batch_size ]
        
        temp_image = []
        temp_label = []
        for path in batch :
            new_arr , mask = self.get_image(path)
            temp_image.append(new_arr)
            temp_label.append(mask/255.0)
        return tf.convert_to_tensor(np.array(temp_image),dtype = 'float32'), tf.convert_to_tensor(np.array(temp_label),dtype = 'float32')   

In [ ]:
import plotly.graph_objects as go

def plot_train_valid_curve(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(df))),y=df['loss'],mode='lines',name = 'Train_loss'))
    
    fig.add_trace(go.Scatter(x=list(range(len(df))),y = df['val_loss'],mode ='lines',name = 'val_loss'))
    
    fig.add_trace(go.Scatter(x=list(range(len(df))),y = df['accuracy'],mode='lines',name = 'Train_accuracy'))
    
    fig.add_trace(go.Scatter(x=list(range(len(df))),y = df['val_accuracy'],mode='lines',name = 'val_accuracy'))
    
    fig.show()
    
def plot_predicted(x_test,valid,pred):
        fig = plt.figure(figsize=(15,7))
        plt.subplot(1,3,1)
        plt.title('Actual Image')
        plt.imshow(x_test)
        
        plt.subplot(1,3,2)
        plt.title('Actual mask')
        plt.imshow(valid)
        
        plt.subplot(1,3,3)
        plt.title('Predicted mask')
        plt.imshow(pred)

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(lst_files)

In [ ]:
print(len(train))
print(len(valid))

## Creating training and validation datasets

In [ ]:
train_ds = create_ds(lst_files=train, batch_size = BATCH_SIZE) 
valid_ds= create_ds(lst_files=valid, batch_size = BATCH_SIZE)

In [ ]:
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
CHANNELS = 3

## Defining the model

In [ ]:
def Unet(IMAGE_HEIGHT , IMAGE_WIDTH,CHANNEL):
    inputs = keras.Input(shape = (IMAGE_HEIGHT, IMAGE_WIDTH, CHANNEL))
    conv_1 = keras.layers.Conv2D(filters = 16, kernel_size = 3, padding='same', activation='relu')(inputs)
    conv_1 = keras.layers.Conv2D(filters = 16, kernel_size = 3, padding='same', activation='relu')(conv_1)
    # conv_1

    conv_2 = keras.layers.MaxPool2D(pool_size = (2,2))(conv_1)
    conv_2 = keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same', activation='relu')(conv_2)
    conv_2 = keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same', activation='relu')(conv_2)

    conv_3 = keras.layers.MaxPool2D(pool_size = (2,2))(conv_2)
    conv_3 = keras.layers.Conv2D(filters = 64, kernel_size = 3, padding='same', activation='relu')(conv_3)
    conv_3 = keras.layers.Conv2D(filters = 64, kernel_size = 3, padding='same', activation='relu')(conv_3)

    conv_4 = keras.layers.MaxPool2D(pool_size = (2,2))(conv_3)
    conv_4 = keras.layers.Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu')(conv_4)
    conv_4 = keras.layers.Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu')(conv_4)

    conv_5 = keras.layers.MaxPool2D(pool_size = (2,2))(conv_4)
    conv_5 = keras.layers.Conv2D(filters = 256, kernel_size = 3, padding='same', activation='relu')(conv_5)
    conv_5 = keras.layers.Conv2D(filters = 256, kernel_size = 3, padding='same', activation='relu')(conv_5)
    
    conv_6 = keras.layers.Conv2DTranspose(filters = 128, kernel_size = 2, strides = 2, padding='same')(conv_5)
    conv_6 = keras.layers.concatenate([conv_4, conv_6])
    conv_6 = keras.layers.Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu')(conv_6)
    conv_6 = keras.layers.Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu')(conv_6)
    
    conv_6 = keras.layers.Conv2DTranspose(filters = 128, kernel_size = 2, strides = 2, padding='same')(conv_5)
    conv_6 = keras.layers.concatenate([conv_4, conv_6])
    conv_6 = keras.layers.Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu')(conv_6)
    conv_6 = keras.layers.Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu')(conv_6)

    conv_7 = keras.layers.Conv2DTranspose(filters = 64, kernel_size = 2, strides = 2, padding='same')(conv_6)
    conv_7 = keras.layers.concatenate([conv_3, conv_7])
    conv_7 = keras.layers.Conv2D(filters = 64, kernel_size = 3, padding='same', activation='relu')(conv_7)
    conv_7 = keras.layers.Conv2D(filters = 64, kernel_size = 3, padding='same', activation='relu')(conv_7)

    conv_8 = keras.layers.Conv2DTranspose(filters = 32, kernel_size = 2, strides = 2, padding='same')(conv_7)
    conv_8 = keras.layers.concatenate([conv_2, conv_8])
    conv_8 = keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same', activation='relu')(conv_8)
    conv_8 = keras.layers.Conv2D(filters = 32, kernel_size = 3, padding='same', activation='relu')(conv_8)


    conv_9 = keras.layers.Conv2DTranspose(filters = 16, kernel_size = 2, strides = 2, padding='same')(conv_8)
    conv_9 = keras.layers.concatenate([conv_1, conv_9])
    conv_9 = keras.layers.Conv2D(filters = 16, kernel_size = 3, padding='same', activation='relu')(conv_9)
    conv_9 = keras.layers.Conv2D(filters = 16, kernel_size = 3, padding='same', activation='relu')(conv_9)

    output = keras.layers.Conv2D(filters = 1, kernel_size = 3, padding='same', activation='relu')(conv_9)
    model = keras.Model(inputs, output)
    return model
    

In [ ]:
model = Unet(IMAGE_HEIGHT , IMAGE_WIDTH , CHANNELS)
model.summary()

In [ ]:
x_train_slices = []
y_train_slices = []

x_test_slices = []
y_test_slices = []

for(train,train_label) in tqdm(train_ds):
    for x_train,y_train in zip(train,train_label):
        x_train_slices.append(x_train.numpy())
        y_train_slices.append(y_train.numpy())
        
for (valid,valid_label) in tqdm(valid_ds):
    for x_test , y_test in zip(valid,valid_label):
        
        x_test_slices.append(x_test.numpy())
        y_test_slices.append(y_test.numpy())

In [ ]:
train_tensor_ds = tf.data.Dataset.from_tensor_slices((x_train_slices, y_train_slices))
valid_tensor_ds = tf.data.Dataset.from_tensor_slices((x_test_slices, y_test_slices))

In [ ]:
BATCH_SIZE = 64
train_tensor_ds = train_tensor_ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
valid_tensor_ds = valid_tensor_ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
def scheduler(epoch):
    if epoch <50:
        return 0.001
    
    elif (50<= epoch and epoch <=70 ):
        return 0.0001
    
    elif (epoch>70):
        return 0.0001 
call_back = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=True)

In [ ]:
model = Unet(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

start = time.time()

history = model.fit(train_tensor_ds, validation_data = valid_tensor_ds, epochs = 100, callbacks = [call_back], verbose=False)

end = time.time()

print(f'Time required for the execution is {end - start}')

model.save_weights('./weights')

In [ ]:
df1 = pd.DataFrame(history.history)
plot_train_valid_curve(df1)

In [ ]:
pred = model.predict(np.array(x_test_slices))
pred_t = (pred >0.5).astype('uint8')

for plot_image in range(3):
    random_image = np.random.randint(0, len(x_test_slices))
    plot_predicted(np.array(x_test_slices)[random_image], np.array(y_test_slices)[random_image],  pred_t[random_image])